In [ ]:
# Uncomment if necessary

In [9]:
#!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [ ]:
#!pip install altair

In [10]:
import h2o
from h2o.estimators import (
    H2OGeneralizedLinearEstimator, 
    H2ORandomForestEstimator, 
    H2OGradientBoostingEstimator, 
    H2ONaiveBayesEstimator,
    H2OStackedEnsembleEstimator,
    H2ODeepLearningEstimator

)
from h2o.frame import H2OFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 days 0 hours 19 mins
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,2 months and 8 days
H2O_cluster_name:,H2O_from_python_vladi_8uz4d9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.897 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


### GLOBAL PRESETS

In [12]:
import warnings
warnings.filterwarnings('ignore')

TEST_SIZE = 0.2

Throughout the project we reference many times the paper: **Practical considerations for specifying a super learner**
https://arxiv.org/pdf/2204.06139

### DATA LOADING AND PREPROCESSING

In [15]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import pandas as pd

In [16]:
spam_data = fetch_openml(data_id=44, as_frame=True)
spam_df = spam_data.frame

X = spam_df.iloc[:, :-1]  # All columns except the last are features
y = spam_df.iloc[:, -1]   # The last column is the target (spam or not)


y = y.astype(int)

# Split the dataset into training (80%) and testing (20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

# We split the temporary dataset into training and test
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=TEST_SIZE, random_state=42)

# We use validation here as test data to compare the individual stacks to not spoil the final test data
h2o_train = H2OFrame(pd.DataFrame(X_train).assign(label=y_train.values))
h2o_val = H2OFrame(pd.DataFrame(X_val).assign(label=y_val.values))
h2o_test = H2OFrame(pd.DataFrame(X_test).assign(label=y_test.values))

# Conversion of target columns to categorical
h2o_train['label'] = h2o_train['label'].asfactor()
h2o_val['label'] = h2o_val['label'].asfactor()
h2o_test['label'] = h2o_test['label'].asfactor()

print("Training set size:", h2o_train.nrows)
print("Validation set size:", h2o_val.nrows)
print("Testing set size:", h2o_test.nrows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Training set size: 2944
Validation set size: 736
Testing set size: 921


#### Is SPAM class underepresented?

In [18]:
class_1 = len(spam_df[spam_df['class'] == '1'])
class_0 = len(spam_df[spam_df['class'] == '0'])
print(f"Records containing spam: {class_1}")
print(f"Records not containing spam: {class_0}")

Records containing spam: 1813
Records not containing spam: 2788


#### Computing the effective sample size n_eff (from paper)

We have binary data, the prevalence of Y is **p=class_1 / total_size**, subsequently **n_rare=n*min(p, 1-p)**, and finally **n_eff=min(n, 5*n_rare)** 

In [20]:
n = len(spam_df)

p = class_1 / n
n_rare = n * min(p, 1-p)
n_eff = min(n, 5*n_rare)
n_eff

4601

#### Computing the V for V-fold cross-validation
Since n_eff >= 500 but not >= 5000 we should select a value between 20 and 10. We take in account that n_eff is closer to 5000 and so we focus on V slightly higher than 10.

In [22]:
N_FOLDS = 12

### BASE LEARNERS - TRAINING & EVALUATION

In [24]:
def train_evaluate_stack(base_learners, metalearner, h2o_train, h2o_test, X_train):
    
    # TRAIN BASE LEARNERS
    print("\n>>> Training base learners:\n")
    for name, learner in base_learners.items():
        print(f"    Training {name} with {N_FOLDS}-fold cross-validation...")
        learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

    super_learner = H2OStackedEnsembleEstimator(
        base_models=list(base_learners.values()),
        metalearner_algorithm=metalearner
    )
    # TRAIN THE METALEARNER
    print("\n>>> Training super learner:\n")
    super_learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

    # EVAL BASE LEARNERS
    print("\n>>> Base learners' results:\n")
    results = {}
    for name, learner in base_learners.items():
        performance = learner.model_performance(test_data=h2o_test)
        f1_score = performance.F1()[0][1]  
        auc_pr = performance.aucpr()      
        accuracy = performance.accuracy()[0][1]
        results[name] = accuracy
        results[name] = {"F1-Score": f1_score, "AUC-PR": auc_pr, "Accuracy": accuracy}
        print(f"    {name} - F1-Score: {f1_score:.4f}, AUC-PR: {auc_pr:.4f}, Accuracy (Test Set): {accuracy:.4f}")
    
    # EVAL THE METALEARNER
    print("\n>>> Metalearner's results:\n")
    super_performance = super_learner.model_performance(test_data=h2o_test)
    super_accuracy = super_performance.accuracy()[0][1]
    super_f1 = super_performance.F1()[0][1]  
    super_auc_pr = super_performance.aucpr()  
    # print(f"\n    Super Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f} | Super Learner Accuracy: {super_accuracy:.4f}")
    
    
    # print("\nFinal Results Comparison:")
    # for name, metrics in results.items():
    #     print(f"{name} - F1-Score: {metrics['F1-Score']:.4f}, AUC-PR: {metrics['AUC-PR']:.4f}, Accuracy: {metrics['Accuracy']:.4f}")
        
    print(f"    Super Learner - F1-Score: {super_f1:.4f}, AUC-PR: {super_auc_pr:.4f}, Accuracy: {super_accuracy:.4f}")
    return {"F1-Score": super_f1, "AUC-PR": super_auc_pr, "Accuracy": super_accuracy}


# Ablation studies

In the following we tried a more methodological way of building the stack. 
We tried two approaches and evaluated their effects on the final test metrics:

**1) Building the stack from simpler models adding more complex ones:**

In this method we start from a base consisting of simple models which we assume would capture the main / most general pattern in the data.
Afterwards we gradually try adding more complex models to extend the stack capabilities to capture more finer intricacies and more complex (perhaps non-linear) relationships in the data and we observe the effect on the test metrics.


**1) Building the stack from more complex models adding more general/simple ones:**
In this method we start from a base consisting of more complex models which we assume would capture the complex relationships in data well and then
we try to bring down the variance by adding simpler models that don't overfit to the data so much.



### A more efficient variant would be training each model only once in case it is present in multiple combinations.

Due to the tradeoff between the scope of this project and time capabilities we perform only superficial overview. If the problem would be a topic of major research where the time needed to search the vast hypothesis space is available, we would suggest performing more extensive per-class tests with higher hyperparameter sampling granularity to better observe how they affect the models performance.

In [27]:
simple_to_complex01 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
}

simple_to_complex02 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex04= {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex05 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex06 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex07 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                    ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                ),   
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex08 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex09 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex10 = {                   
    "LogisticRegression_binomial": H2OGeneralizedLinearEstimator(
                        family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

gradient_boosting_stacks = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [28]:
comparative_results_gb = dict()

for i, stack in enumerate(gradient_boosting_stacks):
    comparative_results_gb[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_val, X_train)


>>> Training base learners:

    Training LogisticRegression_binomial with 12-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    LogisticRegression_binomial - F1-Score: 0.9160, AUC-PR: 0.9554, Accuracy (Test Set): 0.9334

>>> Metalearner's results:

    Super Learner - F1-Score: 0.9160, AUC-PR: 0.9554, Accuracy: 0.9334

>>> Training base learners:

    Training LogisticRegression_binomial with 12-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
    Training GradientBoosting_10trees with 12-fold cross-validation...
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |████████████████

In [29]:
comparative_results_gb

{0: {'F1-Score': 0.9159519725557461,
  'AUC-PR': 0.9553716837388596,
  'Accuracy': 0.9334239130434783},
 1: {'F1-Score': 0.9285714285714286,
  'AUC-PR': 0.9777966725342202,
  'Accuracy': 0.9429347826086957},
 2: {'F1-Score': 0.938566552901024,
  'AUC-PR': 0.981637595362014,
  'Accuracy': 0.9510869565217391},
 3: {'F1-Score': 0.9462738301559792,
  'AUC-PR': 0.983060568473683,
  'Accuracy': 0.9578804347826086},
 4: {'F1-Score': 0.9462738301559792,
  'AUC-PR': 0.9831438722446287,
  'Accuracy': 0.9578804347826086},
 5: {'F1-Score': 0.927487352445194,
  'AUC-PR': 0.9722495829289366,
  'Accuracy': 0.9415760869565217},
 6: {'F1-Score': 0.9285714285714286,
  'AUC-PR': 0.9777950552352713,
  'Accuracy': 0.9429347826086957},
 7: {'F1-Score': 0.9446366782006922,
  'AUC-PR': 0.985264566284969,
  'Accuracy': 0.9565217391304348},
 8: {'F1-Score': 0.9251700680272109,
  'AUC-PR': 0.967172674862178,
  'Accuracy': 0.9402173913043478},
 9: {'F1-Score': 0.9411764705882352,
  'AUC-PR': 0.9784765422527536,
 

In [30]:
simple_to_complex01 = {                   
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
}

simple_to_complex02 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
}

simple_to_complex03 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex04= {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_20trees": H2OGradientBoostingEstimator(
    ntrees=20, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex05 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                        ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                    ),   
    "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex06 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex07 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees": H2OGradientBoostingEstimator(
                    ntrees=10, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
                ),   
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
}


simple_to_complex08 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
        "GradientBoosting_50trees": H2OGradientBoostingEstimator(
    ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}


simple_to_complex09 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

simple_to_complex10 = {                   
"NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True), 
    "GradientBoosting_10trees_unbounded_D": H2OGradientBoostingEstimator(
        ntrees=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
    ),
    "GradientBoosting_20trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=20, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
    "GradientBoosting_50trees_unbounded_D": H2OGradientBoostingEstimator(
    ntrees=50, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True
),
}

gradient_boosting_stacks_nbayes = [
                        simple_to_complex01,
                        simple_to_complex02,
                        simple_to_complex03,
                        simple_to_complex04,
                        simple_to_complex05,
                        simple_to_complex06,
                        simple_to_complex07,
                        simple_to_complex08,
                        simple_to_complex09,
                        simple_to_complex10
]

In [31]:
comparative_results_gb_nbayes = dict()

for i, stack in enumerate(gradient_boosting_stacks_nbayes):
    comparative_results_gb_nbayes[i] = train_evaluate_stack(stack, "glm", h2o_train, h2o_val, X_train)


>>> Training base learners:

    Training NaiveBayes with 12-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base learners' results:

    NaiveBayes - F1-Score: 0.8303, AUC-PR: 0.7914, Accuracy (Test Set): 0.8628

>>> Metalearner's results:

    Super Learner - F1-Score: 0.8303, AUC-PR: 0.7895, Accuracy: 0.8628

>>> Training base learners:

    Training NaiveBayes with 12-fold cross-validation...
naivebayes Model Build progress: |███████████████████████████████████████████████| (done) 100%
    Training GradientBoosting_10trees with 12-fold cross-validation...
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%

>>> Training super learner:

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%

>>> Base l

### RESULTS COMPARISON BETWEEN LOG REGRESSION AND NAIVE BAYES

In [33]:
comparative_results_gb

{0: {'F1-Score': 0.9159519725557461,
  'AUC-PR': 0.9553716837388596,
  'Accuracy': 0.9334239130434783},
 1: {'F1-Score': 0.9285714285714286,
  'AUC-PR': 0.9777966725342202,
  'Accuracy': 0.9429347826086957},
 2: {'F1-Score': 0.938566552901024,
  'AUC-PR': 0.981637595362014,
  'Accuracy': 0.9510869565217391},
 3: {'F1-Score': 0.9462738301559792,
  'AUC-PR': 0.983060568473683,
  'Accuracy': 0.9578804347826086},
 4: {'F1-Score': 0.9462738301559792,
  'AUC-PR': 0.9831438722446287,
  'Accuracy': 0.9578804347826086},
 5: {'F1-Score': 0.927487352445194,
  'AUC-PR': 0.9722495829289366,
  'Accuracy': 0.9415760869565217},
 6: {'F1-Score': 0.9285714285714286,
  'AUC-PR': 0.9777950552352713,
  'Accuracy': 0.9429347826086957},
 7: {'F1-Score': 0.9446366782006922,
  'AUC-PR': 0.985264566284969,
  'Accuracy': 0.9565217391304348},
 8: {'F1-Score': 0.9251700680272109,
  'AUC-PR': 0.967172674862178,
  'Accuracy': 0.9402173913043478},
 9: {'F1-Score': 0.9411764705882352,
  'AUC-PR': 0.9784765422527536,
 

In [34]:
comparative_results_gb_nbayes

{0: {'F1-Score': 0.8302521008403362,
  'AUC-PR': 0.7894555265975739,
  'Accuracy': 0.8627717391304348},
 1: {'F1-Score': 0.9193825042881646,
  'AUC-PR': 0.9727718440346178,
  'Accuracy': 0.936141304347826},
 2: {'F1-Score': 0.9409780775716695,
  'AUC-PR': 0.9788133400682251,
  'Accuracy': 0.9524456521739131},
 3: {'F1-Score': 0.9475465313028766,
  'AUC-PR': 0.9838830527595532,
  'Accuracy': 0.9578804347826086},
 4: {'F1-Score': 0.9475465313028766,
  'AUC-PR': 0.9838812882449601,
  'Accuracy': 0.9578804347826086},
 5: {'F1-Score': 0.9119718309859155,
  'AUC-PR': 0.964425069604786,
  'Accuracy': 0.9320652173913043},
 6: {'F1-Score': 0.9193825042881646,
  'AUC-PR': 0.9728081571984101,
  'Accuracy': 0.936141304347826},
 7: {'F1-Score': 0.9471890971039182,
  'AUC-PR': 0.9848699071120667,
  'Accuracy': 0.9578804347826086},
 8: {'F1-Score': 0.9212730318257957,
  'AUC-PR': 0.9598894331390697,
  'Accuracy': 0.936141304347826},
 9: {'F1-Score': 0.9425675675675677,
  'AUC-PR': 0.9774254151742999,

### LEGEND EXPLANATION
#### LR - Logistic Regression
#### GBX - Gradient Boosting X trees (max depth=10)
#### GBXu - Gradient Boosting X trees (unbounded depth)
#### NB - Naive Bayes

In [41]:
labels = ["LR", "LR_GB10", "LR_GB10_GB20", "LR_GB10_GB20_GB50", "LR_GB10_GB50", "LR_GB10u", "LR_GB10_GB10u", "LR_GB50_GB10u", "LR_GB10u_GB20u", "LR_GB10u_GB20u_GB50u"]

data = []
for label, record in zip(labels, comparative_results_gb.values()):
    record["Configuration"] = label
    data.append(record)
data
df = pd.DataFrame(data)

In [43]:
import altair as alt

f1_chart = alt.Chart(df).mark_point(fill="blue").encode(
    y=alt.Y('Configuration', sort="-x"),
    x=alt.X('F1-Score').scale(zero=False)
).properties(width=150)

aucpr_chart = alt.Chart(df).mark_point(fill="orange", stroke="orange").encode(
    y=alt.Y('Configuration', sort="-x"),
    x=alt.X('AUC-PR').scale(zero=False)
).properties(width=150)

accuracy_chart = alt.Chart(df).mark_point(fill="purple", stroke="purple").encode(
    y=alt.Y('Configuration', sort="-x"),
    x=alt.X('Accuracy').scale(zero=False)
).properties(width=150)
aucpr_chart | f1_chart | accuracy_chart

alt.HConcatChart(...)

In [45]:
labels = ["NB", "NB_GB10", "NB_GB10_GB20", "NB_GB10_GB20_GB50", "NB_GB10_GB50", "NB_GB10u", "NB_GB10_GB10u", "NB_GB50_GB10u", "NB_GB10u_GB20u", "NB_GB10u_GB20u_GB50u"]

data = []
for label, record in zip(labels, comparative_results_gb_nbayes.values()):
    record["Configuration"] = label
    data.append(record)
data
df = pd.DataFrame(data)

In [47]:
f1_chart = alt.Chart(df).mark_point(fill="blue").encode(
    y=alt.Y('Configuration', sort="-x"),
    x=alt.X('F1-Score').scale(zero=False)
).properties(width=150)

aucpr_chart = alt.Chart(df).mark_point(fill="orange", stroke="orange").encode(
    y=alt.Y('Configuration', sort="-x"),
    x=alt.X('AUC-PR').scale(zero=False)
).properties(width=150)

accuracy_chart = alt.Chart(df).mark_point(fill="purple", stroke="purple").encode(
    y=alt.Y('Configuration', sort="-x"),
    x=alt.X('Accuracy').scale(zero=False)
).properties(width=150)
aucpr_chart | f1_chart | accuracy_chart

alt.HConcatChart(...)